# Deep Learning on IBM Stocks

## The Data

We choose to analyse IBM history stock data which include about 13K records from the last 54 years. [From the year 1962 to this day]
Each record contains:  
- Open price: The price in which the market in that month started at.
- Close price: The price in which the market in that month closed at.
- High Price: The max price the stock reached within the month.
- Low price: The min price the stock reached within the month.
- Volume: The max price the stock reached within the month.
- [Adjacent close price](https://marubozu.blogspot.co.il/2006/09/how-yahoo-calculates-adjusted-closing.html).  
- Date: Day, Month and Year.

The main challenges of this project are: 
- The limited data within a market that is changed by wide variety of things. In particular, things that we don't see in the raw data, like special accouncments on new technology.
- The historic data of stocks in a particular situation doesn't necessarily resolve the same outcome in the exact same situation a few years later.
- We wondered whether it is possible to actually find some features that will give us better accuracy than random.  

This project is interesting because as everybody knows deep learning solved tasks that considered difficult even with pretty basic deep learning features.  


And of course, If we find something useful when it comes to stock then good prediction = profit.

In [63]:
from pandas_datareader.data import DataReader
from datetime import datetime
import os
import pandas as pd
import random
import numpy as np
from keras.models import Sequential
from keras.layers.recurrent import LSTM,GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier

from keras.utils.np_utils import to_categorical

#### Load or Download the data

In [64]:
def get_data_if_not_exists(force=False):
    if os.path.exists("./data/ibm.csv") and not force:
        return pd.read_csv("./data/ibm.csv")
    else:
        if not os.path.exists("./data"):
            os.mkdir("data")
        ibm_data = DataReader('IBM', 'yahoo', datetime(1950, 1, 1), datetime.today())
        pd.DataFrame(ibm_data).to_csv("./data/ibm.csv")
        return pd.DataFrame(ibm_data).reset_index()

## Data Exploration

In [65]:
print "loading the data"
data = get_data_if_not_exists(force=True)
print "done loading the data"

loading the data
done loading the data


In [66]:
print "data columns names: %s"%data.columns.values

data columns names: ['Date' 'Open' 'High' 'Low' 'Close' 'Volume' 'Adj Close']


In [67]:
print data.shape
data.head()

(13742, 7)


,Date,Open,High,Low,Close,Volume,Adj Close
0,1962-01-02,578.499734,578.499734,572.000241,572.000241,387200,2.300695
1,1962-01-03,572.000241,576.999736,572.000241,576.999736,288000,2.320804
2,1962-01-04,576.999736,576.999736,570.999742,571.250260,256000,2.297679
3,1962-01-05,570.500243,570.500243,558.999753,560.000253,363200,2.252429
4,1962-01-08,559.500003,559.500003,545.000267,549.500263,544000,2.210196


#### Data exploration highlights:
- The data contains 13,733 records.
- Each record reprsent one specific day.
- Each record contain: Date, Open, High, Low, Close, Volume and Adj Close.

## Feature Extraction and Data Pre-processing.

#### The features are:
1. Open price within the day.
1. Highest price within the day.
1. Lowest price within the day.
1. Close price within the day.
1. Adj Close.
1. Raise percentage.
1. Spread.
1. Up Spread.
1. Down Spread.
1. Absolute Difference between Close and Previous day close.
1. Absolute Difference between Open and Previous day open.
1. Absolute Difference between High and Previous day high.
1. Absolute Difference between low and Previous day low.
1. For each day we've also added a 7 previous day sliding window containing all of the above.
1. 1 When the stock price raised for that day, 0 When the stock price didn't raise.

In [68]:
for i in range(1,len(data)):
    prev = data.iloc[i-1]
    data.set_value(i,"prev_close",prev["Close"])

In [69]:
data["up/down"] = (data["Close"] - data["prev_close"]) > 0

In [70]:
data["raise_percentage"] = (data["Close"] - data["prev_close"])/data["prev_close"]

In [71]:
data["spread"] = abs(data["High"]-data["Low"])

In [72]:
data["up_spread"] = abs(data["High"]-data["Open"])

In [73]:
data["down_spread"] = abs(data["Open"]-data["Low"])

In [74]:
# import re
for i in range(1,len(data)):
    prev = data.iloc[i-1]
    data.set_value(i,"prev_open",prev["Open"])
    data.set_value(i,"prev_high",prev["High"])
    data.set_value(i,"prev_low",prev["Low"])
#     data.set_value(i,"month",re.findall("[1-9]+", str(data.Date[i]))[2])
#     data.set_value(i,"year",re.findall("[1-9]+", str(data.Date[i]))[0])
    
#     prev = data.iloc[i-2]
#     data.set_value(i,"prev_prev_open",prev["Open"])
#     data.set_value(i,"prev_prev_high",prev["High"])
#     data.set_value(i,"prev_prev_low",prev["Low"])
#     data.set_value(i,"prev_prev_close",prev["Close"])

data["close_diff"] = abs(data["Close"] - data["prev_close"])
# data["close_diff"] = data["Close"] - data["prev_close"]
# data["close_diff"] = abs(data["Close"] / data["prev_close"])
data["open_diff"] = abs(data["Open"] - data["prev_open"])
# data["open_diff"] = data["Open"] - data["prev_open"]
# data["open_diff"] = abs(data["Open"] / data["prev_open"])
data["high_diff"] = abs(data["High"] - data["prev_high"])
# data["high_diff"] = data["High"] - data["prev_high"]
# data["high_diff"] = abs(data["High"] / data["prev_high"])
data["low_diff"] = abs(data["Low"] - data["prev_low"])
# data["low_diff"] = data["Low"] - data["prev_low"]
# data["low_diff"] = abs(data["Low"] / data["prev_low"])

# data["prev_prev_close_diff"] = (data["Close"] - data["prev_prev_close"])
# data["prev_prev_raise_percentage"] = (data["Close"] - data["prev_prev_close"])/data["prev_prev_close"]
# data["prev_prev_open_diff"] = (data["Open"] - data["prev_prev_open"])
# data["prev_prev_high_diff"] = (data["High"] - data["prev_prev_high"])
# data["prev_prev_low_diff"] = (data["Low"] - data["prev_prev_low"])
# data["open_close_mean"] = (data["Open"] + data["Close"])/2

In [75]:
# removing the first record because have no previuse record therefore can't know if up or down
data = data[1:]
data.describe()

,Open,High,Low,Close,Volume,Adj Close,prev_close,raise_percentage,spread,up_spread,down_spread,prev_open,prev_high,prev_low,close_diff,open_diff,high_diff,low_diff
count,13741.000000,13741.000000,13741.000000,13741.000000,1.374100e+04,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000,13741.000000
mean,190.008158,191.603641,188.511639,190.033304,4.887131e+06,42.262355,190.063239,0.000130,3.092003,1.595484,1.496519,190.038605,191.634035,188.541637,2.015429,1.945130,1.743916,1.821492
std,132.087441,132.872340,131.418096,132.095695,4.577546e+06,51.494861,132.135641,0.019016,2.524495,1.926574,1.955207,132.128765,132.913067,131.458581,4.574082,4.469844,4.481142,4.526188
min,41.000000,41.750000,40.625000,41.000000,0.000000e+00,1.231153,41.000000,-0.749178,0.000000,0.000000,0.000000,41.000000,41.750000,40.625000,0.000000,0.000000,0.000000,0.000000
25%,97.519997,98.500000,96.500000,97.500000,1.182400e+06,5.944442,97.500000,-0.007973,1.500000,0.375000,0.270004,97.519997,98.500000,96.500000,0.500000,0.500000,0.379997,0.400002
50%,128.125000,129.250000,127.199997,128.250000,4.168400e+06,16.215748,128.250000,0.000000,2.375000,1.000000,0.875000,128.125000,129.250000,127.199997,1.180000,1.125000,1.000000,1.000000
75%,263.750092,266.000000,261.750092,263.750092,6.963100e+06,71.161487,263.875008,0.008315,3.875046,2.029999,1.999498,263.750092,266.000000,261.750092,2.499848,2.375046,2.062500,2.187500
max,649.000015,649.874802,645.500031,649.000015,6.944470e+07,197.047189,649.000015,0.131636,42.000031,28.500009,42.000031,649.000015,649.874802,645.500031,308.499985,309.000015,311.500015,312.999992


In [76]:
MAX_WINDOW = 5

In [77]:
def extract_features(items):
    return [[item[1], item[2], item[3], item[4],
            item[5], item[6], item[9], item[10],
            item[11], item[12], item[16], item[17],
            item[18], item[19], 1] 
            
            if item[8] 
            
            else 
           [item[1], item[2], item[3], item[4],
            item[5], item[6], item[9], item[10],
            item[11], item[12], item[16], item[17],
            item[18], item[19], 0] 
            
            for item in items]
                

# def extract_features(items):
#     return [[item[12],item[11],item[10],item[9], 1] if item[8] else [item[12],item[11],item[10],item[9], -1] for item in items]

In [78]:
def extract_expected_result(item):
    return 1 if item[8] else 0

In [79]:
def generate_input_and_outputs(data,extractFeaturesFunc=extract_features,expectedResultFunc=extract_expected_result):
    step = 1
    inputs = []
    outputs = []
    for i in range(0, len(data) - MAX_WINDOW, step):
        inputs.append(extractFeaturesFunc(data.iloc[i:i + MAX_WINDOW].as_matrix()))
        outputs.append(expectedResultFunc(data.iloc[i + MAX_WINDOW].as_matrix()))
    return inputs, outputs

In [20]:
print "generating model input and outputs"
X, y = generate_input_and_outputs(data)
print "done generating input and outputs"

generating model input and outputs
done generating input and outputs


In [21]:
y = to_categorical(y)

### Splitting the data to train and test

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [23]:
X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=0.15)

## Configuration of the deep learning models

In [1297]:
models = []

In [1298]:
layer_output_size1 = 128
layer_output_size2 = 128
output_classes = len(y[0])
percentage_of_neurons_to_ignore = 0.2


model = Sequential()
model.add(LSTM(layer_output_size1, return_sequences=True, input_shape=(MAX_WINDOW, len(X[0][0]))))
model.add(Dropout(percentage_of_neurons_to_ignore))
model.add(LSTM(layer_output_size2, return_sequences=False))
model.add(Dropout(percentage_of_neurons_to_ignore))
model.add(Dense(output_classes))
model.add(Activation('softmax'))
model.alg_name = "LSTM"
model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='rmsprop')
models.append(model)

model = Sequential()
model.add(SimpleRNN(layer_output_size1, return_sequences=True, input_shape=(MAX_WINDOW, len(X[0][0]))))
model.add(Dropout(percentage_of_neurons_to_ignore))
model.add(SimpleRNN(layer_output_size2, return_sequences=False))
model.add(Dropout(percentage_of_neurons_to_ignore))
model.add(Dense(output_classes))
model.add(Activation('softmax'))
model.alg_name = "SimpleRNN"
model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='rmsprop')
models.append(model)

model = Sequential()
model.add(GRU(layer_output_size1, return_sequences=True, input_shape=(MAX_WINDOW, len(X[0][0]))))
model.add(Dropout(percentage_of_neurons_to_ignore))
model.add(GRU(layer_output_size2, return_sequences=False))
model.add(Dropout(percentage_of_neurons_to_ignore))
model.add(Dense(output_classes))
model.add(Activation('softmax'))
model.alg_name = "GRU"
model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='rmsprop')
models.append(model)

### Training

In [1299]:
def trainModel(model):
    epochs = 5
    print "Training model %s"%(model.alg_name)
    model.fit(X_train, y_train, batch_size=128, nb_epoch=epochs,validation_data=(X_validation,y_validation), verbose=0)

### Evaluation

In [1300]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

def createSplit(model):
    print 'Adding layer of DecisionTreeClassifier'
#     split_model = RandomForestClassifier()
#     split_model.fit(model.predict(X_validation), y_validation)
    
#     split_model = ExtraTreesClassifier(n_estimators=15, max_depth=None, min_samples_split=2, random_state=0)
#     split_model.fit(model.predict(X_validation), y_validation)
    
#     split_model = DecisionTreeClassifier(max_depth=None, min_samples_split=1, random_state=0)
#     split_model.fit(model.predict(X_validation), y_validation)
    
    split_model = DecisionTreeClassifier()
    split_model.fit(model.predict(X_validation), y_validation)
    return split_model

In [1301]:
def probabilities_to_prediction(record):
    return [1,0] if record[0]>record[1] else [0,1]

In [1307]:
def evaluateModel(model):
    success, success2 = 0,0
    predicts = model.predict(X_test)
    split_model = createSplit(model)
    for index, record in enumerate(predicts):
        predicted = list(split_model.predict([np.array(record)])[0])
        predicted2 = probabilities_to_prediction(record)
        expected = y_test[index]
        if predicted[0] == expected[0]:
            success += 1
        if predicted2[0] == expected[0]:
            success2 += 1
    accuracy = float(success) / len(predicts)
    accuracy2 = float(success2) / len(predicts)
    print "The Accuracy for %s is: %s" % (model.alg_name, max(accuracy2, accuracy))
    return accuracy

In [1303]:
def train_and_evaluate():
    accuracies = {}
    for model in models:
        trainModel(model)
        acc = evaluateModel(model)
        if model.alg_name not in accuracies:
            accuracies[model.alg_name] = []
        accuracies[model.alg_name].append(acc)
    return accuracies

In [1310]:
acc = train_and_evaluate()

Training model LSTM
Adding layer of DecisionTreeClassifier
The Accuracy for LSTM is: 0.52572815534
Training model SimpleRNN
Adding layer of DecisionTreeClassifier
The Accuracy for SimpleRNN is: 0.526213592233
Training model GRU
Adding layer of DecisionTreeClassifier
The Accuracy for GRU is: 0.52572815534


### Naive algorithm:

We'll choose the most frequent up / down of the stock.

In [1326]:
all_data = data["up/down"].count()
most_frequent = data["up/down"].describe().top
frequency = data["up/down"].describe().freq
acc = float(frequency) / all_data

print 'The most frequent is: %s' % (most_frequent) 
print 'The accuracy of naive algorithm is: ', acc

The most frequent is: False
The accuracy of naive algorithm is:  0.51303524614


## Summary & Evaluation analysis:

#### Evaluation process:
Our evaluation used two different configurations:
1. Raw Deep-Learning algorithm.
1. Deep-Learning algorithm With added layer of DecisionTreeClassifier.

In both cases we used the predictions of the algorithm to create a sequence to tell us whether the stock is going to get up or down. Then we checked it with the actual data and calculated accuracy.

### Results:
The accuracy as stated above is better then a naive algorithm, Not by far, But still better which means that if we follow the algorithm we are actually expected to make profit.

### What next?
As expected it seems like the raw stock data isn't get a high estimation of the stock behavior.
We could try mixing it with information from financial articles and news,  try to take into account related stocks like the sector, S&P500 and  new features,  even checking for a country specific economics laws.

# Creating sequence of close price from the stock data

In [143]:
def extract_features2(items):
    return [[item[4]] for item in items]

In [144]:
def extract_expected_result2(item):
    return [item[4]]

In [147]:
MAX_WINDOW = 5

In [148]:
def train_test_split(data, test_size=0.1):
    """
    This just splits data to training and testing parts
    """
    ntrn = int(round(len(data) * (1 - test_size)))
    X, y = generate_input_and_outputs(data,extract_features2,extract_expected_result2)
    X_train,y_train,X_test, y_test = X[:ntrn],y[:ntrn],X[ntrn:],y[ntrn:]

    return X_train, y_train, X_test, y_test

In [149]:
X_train,y_train, X_test, y_test = train_test_split(data,test_size=0.15)

In [150]:
layer_output_size1 = 128

model = Sequential()
model.add(LSTM(layer_output_size1, return_sequences=False, input_shape=(MAX_WINDOW, len(X[0][0]))))
model.add(Dense(len(y[0]), input_dim=layer_output_size1))
model.add(Activation("linear"))
model.compile(loss="mean_squared_error", optimizer="rmsprop")

In [153]:
model.fit(X_train, y_train, batch_size=700, nb_epoch=500, validation_split=0.15,verbose=1)

Train on 9928 samples, validate on 1752 samples
Epoch 1/300
9928/9928 [==============================] - 3s - loss: 2559.5334 - val_loss: 2.2124
Epoch 2/300
9928/9928 [==============================] - 2s - loss: 2528.4861 - val_loss: 6.9209
Epoch 3/300
9928/9928 [==============================] - 2s - loss: 2498.9034 - val_loss: 2.2409
Epoch 4/300
9928/9928 [==============================] - 2s - loss: 2467.1285 - val_loss: 3.5810
Epoch 5/300
9928/9928 [==============================] - 2s - loss: 2435.1240 - val_loss: 5.8971
Epoch 6/300
9928/9928 [==============================] - 2s - loss: 2405.7564 - val_loss: 3.1488
Epoch 7/300
9928/9928 [==============================] - 2s - loss: 2374.8461 - val_loss: 2.7618
Epoch 8/300
9928/9928 [==============================] - 2s - loss: 2344.0779 - val_loss: 2.0936
Epoch 9/300
9928/9928 [==============================] - 2s - loss: 2313.9590 - val_loss: 2.4264
Epoch 10/300
9928/9928 [==============================] - 2s - loss: 2287.5917 

In [154]:
predicted = model.predict(X_test)

In [155]:
zip(predicted,y_test)

[(array([ 131.11024475]), [128.470001]),
 (array([ 132.18635559]), [124.94000200000001]),
 (array([ 128.33589172]), [125.860001]),
 (array([ 129.93501282]), [125.94000200000001]),
 (array([ 129.75747681]), [123.25]),
 (array([ 127.07476044]), [123.849998]),
 (array([ 128.11689758]), [126.150002]),
 (array([ 130.32107544]), [126.709999]),
 (array([ 130.48796082]), [125.099998]),
 (array([ 128.73202515]), [124.160004]),
 (array([ 128.11016846]), [125.019997]),
 (array([ 129.09962463]), [122.739998]),
 (array([ 126.69590759]), [123.459999]),
 (array([ 127.78218079]), [123.459999]),
 (array([ 127.57756042]), [124.58000200000001]),
 (array([ 128.7129364]), [121.129997]),
 (array([ 125.22927856]), [120.050003]),
 (array([ 124.77232361]), [118.529999]),
 (array([ 123.2224884]), [119.269997]),
 (array([ 123.92341614]), [119.099998]),
 (array([ 123.48387909]), [119.540001]),
 (array([ 123.96127319]), [121.5]),
 (array([ 125.93928528]), [123.879997]),
 (array([ 128.16056824]), [120.400002]),
 (a